# RATIO 2019 - Benchmarking Workshop

In [1]:
%load_ext autoreload
import os
os.environ['CUDA_VISIBLE_DEVICES']='5'

from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
from torch.optim import lr_scheduler
import os
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd 

In [2]:
import pandas as pd
import csv
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize

In [3]:
from sklearn.model_selection import train_test_split
import nltk
nltk.download('wordnet')
def get_train_test_sets(df):
    X = df[['argument1', 'argument2', 'topic']]
    y = df[['is_same_side']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1, shuffle=True)
    return X_train, X_test, y_train, y_test

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Task 1 - Same Side Clasiification



In [4]:
data_cross_path = 'data/same-side-classification/cross-topic/{}.csv'
data_within_path = 'data/same-side-classification/within-topic/{}.csv'

### Load within-topics and cross-topics data

In [5]:
cross_traindev_df = pd.read_csv(data_cross_path.format('training'), 
                                quotechar='"',quoting=csv.QUOTE_ALL,encoding='utf-8',escapechar='\\',doublequote=False,  index_col='id')
cross_test_df =  pd.read_csv(data_cross_path.format('test'), encoding='utf-8',index_col='id')


within_traindev_df =  pd.read_csv(data_within_path.format('training'), 
                                quotechar='"',quoting=csv.QUOTE_ALL,encoding='utf-8',escapechar='\\',doublequote=False, index_col='id')

within_test_df =  pd.read_csv(data_within_path.format('test'), encoding='utf-8',index_col='id')

In [7]:
# Adding a tag for the topics in focus: "gay marriage" and "abortion"
def add_tag(row):
    title = row['topic'].lower().strip()
    if title.find('abortion') > -1 :
        row['tag'] = 'abortion'
    elif title.find('gay marriage') > -1 :
        row['tag'] = 'gay marriage'
    else:
        row['tag'] = 'NA'
    return row

cross_traindev_df = cross_traindev_df.apply(add_tag, axis=1)
cross_test_df = cross_test_df.apply(add_tag, axis=1)

within_traindev_df = within_traindev_df.apply(add_tag, axis=1)
within_test_df = within_test_df.apply(add_tag, axis=1)



In [8]:
print('within train df:', len(within_traindev_df))
print('cross train df:', len(cross_traindev_df))

within train df: 63903
cross train df: 61048


In [9]:
print('within test df:', len(within_test_df))
print('cross test df:', len(cross_test_df))

within test df: 31475
cross test df: 6163


In [10]:
cross_traindev_df.head(n=20)

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
0,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
1,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
2,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,this is my first debate so please just bare wi...,100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
3,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00002-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00002-000,100c174f-2019-04-18T17:33:51Z,True,abortion should be illegal with exceptions,abortion
4,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion
5,there are two reasons why this debate should g...,100c174f-2019-04-18T17:33:51Z-00000-000,this is my first debate so please just bare wi...,100c174f-2019-04-18T17:33:51Z-00000-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion
6,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion
7,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion
8,i will give my opponent a chance to respond.,100c174f-2019-04-18T17:33:51Z-00002-000,this is my first debate so please just bare wi...,100c174f-2019-04-18T17:33:51Z-00002-000,100c174f-2019-04-18T17:33:51Z,False,abortion should be illegal with exceptions,abortion


In [11]:
within_traindev_df.head()

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
85249,"gay marriage devalues marriage, frequency of o...",d2f4b1cd-2019-04-17T11:47:27Z-00063-000,being unaccustomed to gay marriage is no argument,d2f4b1cd-2019-04-17T11:47:27Z-00063-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage
2607,accepted. pro may extend their arguments to th...,2a0d32eb-2019-04-18T11:46:44Z-00004-000,"i""m pro-life. just think about it, your murder...",2a0d32eb-2019-04-18T11:46:44Z-00004-000,2a0d32eb-2019-04-18T11:46:44Z,False,abortion (pro life),abortion
14632,ultrasounds fit well with pro-choice concepts.,475596d3-2019-04-17T11:47:21Z-00031-000,ultrasounds are a procedure any pregnant woman...,475596d3-2019-04-17T11:47:21Z-00031-000,475596d3-2019-04-17T11:47:21Z,True,mandatory ultrasounds before abortions,abortion
29855,abortion should be banned nationally unless it...,a89a42-2019-04-18T19:55:57Z-00003-000,abortion should be banned nationally unless it...,a89a42-2019-04-18T19:55:57Z-00003-000,a89a42-2019-04-18T19:55:57Z,False,abortion should be banned nationally unless it...,abortion
84168,marriage is defined as between a man and woman,d2f4b1cd-2019-04-17T11:47:27Z-00092-000,marriage is celebrated because of the assumpti...,d2f4b1cd-2019-04-17T11:47:27Z-00092-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage


In [12]:
cross_df = cross_traindev_df.drop_duplicates(['argument1'])
within_df = within_traindev_df.drop_duplicates(['argument1'])

In [13]:
print('within train df:', len(cross_df))
print('cross train df:', len(within_df))

within train df: 7828
cross train df: 10508


In [14]:
cross_df.groupby('is_same_side').count()

,argument1,argument1_id,argument2,argument2_id,debate_id,topic,tag
is_same_side,,,,,,,
False,1548,1548,1548,1548,1548,1548,1548
True,6280,6280,6280,6280,6280,6280,6280


In [15]:
within_df.groupby('is_same_side').count()

,argument1,argument1_id,argument2,argument2_id,debate_id,topic,tag
is_same_side,,,,,,,
False,6230,6230,6230,6230,6230,6230,6230
True,4278,4278,4278,4278,4278,4278,4278


### Approach

In [16]:
%autoreload

from siamese import *
import pickle

2019-12-04 21:31:51,570 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmppe5idwk0


100%|██████████| 160000128/160000128 [00:06<00:00, 24436135.61B/s]

2019-12-04 21:31:58,219 copying /tmp/tmppe5idwk0 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2019-12-04 21:31:58,452 removing temp file /tmp/tmppe5idwk0
2019-12-04 21:31:58,616 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmpgh74qgtd


100%|██████████| 21494764/21494764 [00:00<00:00, 26252316.66B/s]

...

2019-12-04 21:32:04,488 copying /tmp/tmpk0cl29y4 to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2019-12-04 21:32:04,605 removing temp file /tmp/tmpk0cl29y4
2019-12-04 21:32:09,358 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-backward--h2048-l1-d0.05-lr30-0.25-20/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmprh4pf712


100%|██████████| 73034575/73034575 [00:05<00:00, 14211393.22B/s]

2019-12-04 21:32:14,678 copying /tmp/tmprh4pf712 to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2019-12-04 21:32:14,777 removing temp file /tmp/tmprh4pf712


#### Embedding training examples using Flair:

In [17]:
X_train, X_dev, y_train, y_dev = get_train_test_sets(cross_df)

In [18]:
print('train:', len(X_train), 'dev:', len(X_dev))

train: 5479 dev: 2349


In [ ]:
x1_train, x2_train, y_train = process_df(X_train, y_train, passing_y=True)
x1_dev, x2_dev, y_dev = process_df(X_dev, y_dev, passing_y=True)

In [54]:
import pickle

pickle.dump(x1_train, open('/workspace/webis20_data/same-stance-detection/x1_train_filter.pickle', 'wb'))
pickle.dump(x2_train, open('/workspace/webis20_data/same-stance-detection/x2_train_filter.pickle', 'wb'))
pickle.dump(y_train, open('/workspace/webis20_data/same-stance-detection/y_train_filter.pickle', 'wb'))

pickle.dump(x1_dev, open('/workspace/webis20_data/same-stance-detection/x1_dev_filter.pickle', 'wb'))
pickle.dump(x2_dev, open('/workspace/webis20_data/same-stance-detection/x2_dev_filter.pickle', 'wb'))
pickle.dump(y_dev, open('/workspace/webis20_data/same-stance-detection/y_dev_filter.pickle', 'wb'))

In [19]:
x1_train = pickle.load(open('/workspace/webis20_data/same-stance-detection/x1_train_filter.pickle', 'rb'))
x2_train = pickle.load(open('/workspace/webis20_data/same-stance-detection/x2_train_filter.pickle', 'rb'))
y_train  = pickle.load(open('/workspace/webis20_data/same-stance-detection/y_train_filter.pickle', 'rb'))

x1_dev = pickle.load(open('/workspace/webis20_data/same-stance-detection/x1_dev_filter.pickle', 'rb'))
x2_dev = pickle.load(open('/workspace/webis20_data/same-stance-detection/x2_dev_filter.pickle', 'rb'))
y_dev  = pickle.load(open('/workspace/webis20_data/same-stance-detection/y_dev_filter.pickle', 'rb'))

#### Train the model:

In [56]:
model = train(25, x1_train, x2_train, y_train, x1_dev, x2_dev, y_dev)

Loaded 5479 training data.
Loaded 2349 development data.
Epoch [1/25], Iter [343/342] Loss: 0.2231
Accuracy of the model on the 5479.0 training images: 72.3672202956744 %%
Accuracy of the model on the 2349.0 development images: 71.56236696466581 %%
Epoch [2/25], Iter [343/342] Loss: 0.2329
Accuracy of the model on the 5479.0 training images: 76.03577295126848 %%
Accuracy of the model on the 2349.0 development images: 75.18092805449128 %%
Epoch [3/25], Iter [343/342] Loss: 0.8772
Accuracy of the model on the 5479.0 training images: 76.63807264099289 %%
Accuracy of the model on the 2349.0 development images: 75.98978288633461 %%
Epoch [4/25], Iter [343/342] Loss: 0.1843
Accuracy of the model on the 5479.0 training images: 80.39788282533308 %%
Accuracy of the model on the 2349.0 development images: 79.4806300553427 %%
Epoch [5/25], Iter [343/342] Loss: 0.4633
Accuracy of the model on the 5479.0 training images: 80.43438583683154 %%
Accuracy of the model on the 2349.0 development images: 7

In [18]:
torch.save(model.state_dict(), '/workspace/webis20_data/same-stance-detection/cross_topic_model_weights')

#### Within-topic experiment:

In [15]:
X_within_train, X_within_dev, y_within_train, y_within_dev = get_train_test_sets(within_traindev_df)

In [16]:
print('train:', len(X_within_train), 'dev:', len(X_within_dev))

train: 44732 dev: 19171


In [ ]:
x1_within_train, x2_within_train, y_within_train = process_df(X_within_train, y_within_train, passing_y=True)

In [21]:
x1_within_dev, x2_within_dev, y_within_dev = process_df(X_within_dev, y_within_dev, passing_y=True)

In [23]:
import pickle

pickle.dump(x1_within_train, open('/workspace/webis20_data/same-stance-detection/x1_within_train.pickle', 'wb'))
pickle.dump(x2_within_train, open('/workspace/webis20_data/same-stance-detection/x2_within_train.pickle', 'wb'))
pickle.dump(y_within_train, open('/workspace/webis20_data/same-stance-detection/y_within_train.pickle', 'wb'))

pickle.dump(x1_within_dev, open('/workspace/webis20_data/same-stance-detection/x1_within_dev.pickle', 'wb'))
pickle.dump(x2_within_dev, open('/workspace/webis20_data/same-stance-detection/x2_within_dev.pickle', 'wb'))
pickle.dump(y_within_dev, open('/workspace/webis20_data/same-stance-detection/y_within_dev.pickle', 'wb'))

In [19]:
x1_within_train = pickle.load(open('/workspace/webis20_data/same-stance-detection/x1_within_train.pickle', 'rb'))
x2_within_train = pickle.load(open('/workspace/webis20_data/same-stance-detection/x2_within_train.pickle', 'rb'))
y_within_train  = pickle.load(open('/workspace/webis20_data/same-stance-detection/y_within_train.pickle', 'rb'))

x1_within_dev = pickle.load(open('/workspace/webis20_data/same-stance-detection/x1_within_dev.pickle', 'rb'))
x2_within_dev = pickle.load(open('/workspace/webis20_data/same-stance-detection/x2_within_dev.pickle', 'rb'))
y_within_dev  = pickle.load(open('/workspace/webis20_data/same-stance-detection/y_within_dev.pickle', 'rb'))

In [24]:
within_model = train(25, x1_within_train, x2_within_train, y_within_train, x1_within_dev, x2_within_dev, y_within_dev)

Loaded 44732 training data.
Loaded 19171 development data.
Epoch [1/25], Iter [2796/2795] Loss: 0.3553
Accuracy of the model on the 44732.0 training images: 77.92184565858894 %%
Accuracy of the model on the 19171.0 development images: 77.01215377392937 %%
Epoch [2/25], Iter [2796/2795] Loss: 0.3713
Accuracy of the model on the 44732.0 training images: 81.20137709022623 %%
Accuracy of the model on the 19171.0 development images: 80.48093474518805 %%
Epoch [3/25], Iter [2796/2795] Loss: 0.2166
Accuracy of the model on the 44732.0 training images: 81.98157918268801 %%
Accuracy of the model on the 19171.0 development images: 81.07558291168954 %%
Epoch [4/25], Iter [2796/2795] Loss: 0.2554
Accuracy of the model on the 44732.0 training images: 82.12241795582581 %%
Accuracy of the model on the 19171.0 development images: 81.47723123467738 %%
Epoch [5/25], Iter [2796/2795] Loss: 0.3628
Accuracy of the model on the 44732.0 training images: 82.22525261557722 %%
Accuracy of the model on the 19171

In [25]:
torch.save(within_model.state_dict(), '/workspace/webis20_data/same-stance-detection/within_topic_model_weights')

### Test Model:

#### Testing cross-topic model

In [78]:
x1_dev = pickle.load(open('/workspace/webis20_data/same-stance-detection/x1_dev_filter.pickle', 'rb'))
x2_dev = pickle.load(open('/workspace/webis20_data/same-stance-detection/x2_dev_filter.pickle', 'rb'))
y_dev  = pickle.load(open('/workspace/webis20_data/same-stance-detection/y_dev_filter.pickle', 'rb'))

In [79]:
test('/workspace/webis20_data/same-stance-detection/cross_topic_model_weights_filter', x1_dev, x2_dev, y_dev)

Loaded 2349 test data.
Accuracy of the model on the 2349.0 testing images: 80.45977011494253 %%


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [62]:
X_cross = cross_test_df[['argument1', 'argument2']]
#X_cross = X_cross.sample(20)
x1_test, x2_test, y_test = process_df(X_cross, passing_y=False)

In [80]:
y_preds = test('/workspace/webis20_data/same-stance-detection/cross_topic_model_weights', x1_test, x2_test, y_test)

Loaded 6163 test data.
Accuracy of the model on the 6163.0 testing images: 69.00859970793445 %%


In [81]:
#Save predictions ...
out_file = open('/workspace/webis20_data/same-stance-detection/cross_topic_test_preds.csv', 'w')
out_file.write('id,label\n')
for i in range(len(y_preds)):
    out_file.write('{},{} \n'.format(cross_test_df.index[i], 'True' if y_preds[i] == 1.0 else 'False'))
out_file.close()

#### Testing within-topic model

In [21]:
x1_wtihin_dev = pickle.load(open('/workspace/webis20_data/same-stance-detection/x1_within_dev.pickle', 'rb'))
x2_within_dev = pickle.load(open('/workspace/webis20_data/same-stance-detection/x2_within_dev.pickle', 'rb'))
y_within_dev  = pickle.load(open('/workspace/webis20_data/same-stance-detection/y_within_dev.pickle', 'rb'))

In [22]:
test('/workspace/webis20_data/same-stance-detection/within_topic_model_weights', x1_wtihin_dev, x2_within_dev, y_within_dev)

Loaded 19171 test data.
Accuracy of the model on the 19171.0 testing images: 82.37441969641647 %%


[0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [7]:
X_within10 = within_test_df[['argument1', 'argument2']][0:10000]

In [ ]:
x1_within_test, x2_within_test, y_within_test = process_df(X_within10, passing_y=False)

In [ ]:
y_preds = test('/workspace/webis20_data/same-stance-detection/within_topic_model_weights', x1_within_test, x2_within_test, y_within_test)

In [33]:
y_preds

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [34]:
#Save predictions ...
out_file = open('/workspace/webis20_data/same-stance-detection/within_topic_test_preds_30k.csv', 'w')
out_file.write('id,label\n')
for i in range(len(y_preds)):
    out_file.write('{},{} \n'.format(within_test_df.index[i], 'True' if y_preds[i] == 1.0 else 'False'))
    
out_file.close()

In [35]:
ls /workspace/webis20_data/same-stance-detection/

cross_topic_model_weights            x2_dev.pickle
cross_topic_model_weights_filter     x2_dev_filter.pickle
cross_topic_test_preds.csv           x2_train.pickle
within_topic_model_weights           x2_train_filter.pickle
within_topic_model_weights_filtered  x2_within_dev.pickle
within_topic_test_preds.csv          x2_within_dev_filtered.pickle
within_topic_test_preds_10k.csv      x2_within_train.pickle
within_topic_test_preds_20k.csv      x2_within_train_filtered.pickle
within_topic_test_preds_30k.csv      y_dev.pickle
x1_dev.pickle                        y_dev_filter.pickle
x1_dev_filter.pickle                 y_train.pickle
x1_train.pickle                      y_train_filter.pickle
x1_train_filter.pickle               y_within_dev.pickle
x1_within_dev.pickle                 y_within_dev_filtered.pickle
x1_within_dev_filtered.pickle        y_within_train.pickle
x1_within_train.pickle               y_within_train_filtered.pickle
x1_within_train_filtered.pickle


In [30]:
pred_10 = list(map(lambda x: x.split(','), 
                   open('/workspace/webis20_data/same-stance-detection/within_topic_test_preds_10k.csv', 'r').read().split('\n')[1:-1]))
pred_20 = list(map(lambda x: x.split(','), 
                   open('/workspace/webis20_data/same-stance-detection/within_topic_test_preds_20k.csv', 'r').read().split('\n')[1:-1]))
pred_30 = list(map(lambda x: x.split(','), 
                   open('/workspace/webis20_data/same-stance-detection/within_topic_test_preds_30k.csv', 'r').read().split('\n')[1:-1]))

In [35]:
len(within_test_df)

31475

In [39]:
out_file = open('/workspace/webis20_data/same-stance-detection/within_topic_test_final.csv', 'w')
out_file.write('id,label\n')
for i in range(len(within_test_df)):
    if i < 10000:
        pred = pred_10[i][1].strip()
    if i >= 10000 and i < 20000:
        pred = pred_20[i-10000][1].strip()
    if i > 20000:
        pred = pred_30[i-20000][1].strip()
        
    out_file.write('{},{} \n'.format(within_test_df.index[i], pred))
    
out_file.close()